# Advent of Code 2016

## Day 22: Grid Computing

In [1]:
from collections import namedtuple
import re

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

### Solution Part One

I start with defining a custom datatype for my nodes, just for convenience and to keep my sanity while coding. If you don't want any methods or state changes, `namedtuple` is great. They sort and hash just like regular tuples, but the fields have names.

In [2]:
Node = namedtuple('Node', ('x', 'y', 'size', 'used', 'avail'))

Mandatory horrific regex to parse the input.

In [3]:
r = re.compile(r'[/a-z\-]+(\d+)-y(\d+)\s+(\d+)T\s+(\d+)T\s+(\d+)T')

Just put all the inputs into a single list.

In [4]:
nodes = []
with open('input.txt') as f:
    for line in f:
        m = r.match(line)
        if m:
            node = Node(*map(int, m.groups()))
            nodes.append(node)

Find the **viable pairs**.

In [5]:
nodes_by_used = sorted(nodes, key=lambda n: n.used)
nodes_by_avail = sorted(nodes, key=lambda n: n.avail)

n_pairs = 0
for a in nodes_by_used:
    if a.used == 0:
        continue
    for b in reversed(nodes_by_avail):
        if a is b:
            continue
        if a.used > b.avail:
            break
        n_pairs += 1

print('Answer Part One:', n_pairs)

Answer Part One: 860


### Solution Part Two

I quickly noticed that - apart from some huge, essentially immobile, nodes and one empty node - all nodes were larger than 80T and contained less than 80T of data. Great! It seems that we can make the same simplification as in the example. Let's print the grid to see what we have.

In [6]:
for node in sorted(nodes, key=lambda n: (n.y, n.x)):
    if node.x == 0:
        print()  # New line
    if node.used > 200:
        print('#', end=' ')
    elif node.used == 0 and node.size > 80:
        print('_', end=' ')
    elif 0 < node.used < 80 and node.size > 80:
        print('.', end=' ')
    else:
        # This should never run if the assumption is correct
        print('?', end=' ')



. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . # # # # # # # # # # # # # # 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . .

By now it is clear that apart from the immobile row, any data fits anywhere. So, first we need to move the hole to `node-x27-y14` to `node-x33-y0`. To do this, we need to move the hole to `node-x20-y11` in order to avoid the immobile row. This takes 7 + 3 = 10 steps. Then the path is clear and we get to `node-x33-y0` after another 13 + 11 = 24 steps. Then move the Goal data to `node-x33-y0`.

From this point, we follow the scheme described in the example. To move the Goal data from `node-x(n)-y0` to `node-x(n-1)-y0`, first move the hole to `node-x(n-1)-y0`, avoiding the goal data and then move the goal data. Each iteration takes 5 steps. All in all 33x5.

Thus, the number of steps needed is 10 + 24 + 1 + 33 x 5 = 200.

In [7]:
10 + 24 + 1 + 33*5

200